In [1]:
# https://ai.google/tools/datasets/google-facial-expression/
import imageio
import os
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm
import time
import numpy as np
import random
import hashlib
from skimage.transform import resize
import math

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import * 
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend as K

In [3]:
train_df = pd.read_csv('FEC_dataset/two_class_ready.csv')
train_df.head()

,Image1,Image2,Image3
0,22d27258ff7e78577c811c5b8187446f.jpg,5806875441d55aa9c1d8a8998d160a82.jpg,bb3f152775a40c53daafab83a43f5254.jpg
1,e5a2d4733125b8aedb18c2d8040868e6.jpg,8ed9229eb118373590a069b774aeeeec.jpg,9d5910b05710e09d2ab12e4d4b224938.jpg
2,e5a2d4733125b8aedb18c2d8040868e6.jpg,53e165101543063e4b4946eb411ebecd.jpg,bad09c8ea230ee5919fcf794ea7f4f4b.jpg
3,e5a2d4733125b8aedb18c2d8040868e6.jpg,0d3252b4ad950e4e875fd2ace92c9641.jpg,9f99820796b92f6566a4639028ad0a00.jpg
4,e5a2d4733125b8aedb18c2d8040868e6.jpg,59d6ef6f3971c8a828fb67b9961dba23.jpg,08d4159f45a1607a0aa00bd6dc75c24f.jpg


In [4]:
def TripletGen(data, root='FEC_dataset/formatted', batch_size=128, rescale=1./255, shape=(224,224)):
    # yields anchor, positive and negative examples in batches of @batch_size
    # output shape is (batch_size, 3, 224, 224, 3)
    n = len(data)
    iterations = math.ceil(n/batch_size)
    print(f"Found {n} triplets in the dataset")
    # the main "yielding" loop
    for i in range(iterations):
        # use start and end as indices to work with
        start, end = i*batch_size, min((i+1)*batch_size, n)
        batch_size = end - start
        
        # yield X each time
        X = np.empty((batch_size, 3, 224, 224, 3))
        
        for j in range(batch_size):
            # X[j] row is a triplet of size (3, 224, 224, 3)
            for t in range(1,4):
                path = os.path.join(root, data[f'Image{t}'][start+j])
                temp = image.load_img( path, target_size = shape )
                X[j, t-1] = image.img_to_array(temp)*rescale
        yield X

In [195]:
# loss functions are defined for batches of triplets over embedding space
def TripletLoss(dist='sqeuclidean', margin='maxplus'):
    
    def curry(inputs, y_pred=None):
        inputs = inputs.astype(np.float32)
        anchor, positive, negative = inputs[:,0, ...], inputs[:,1, ...], inputs[:,2, ...]
        positive_distance = np.square(anchor - positive)
        negativesqeuclidean_distance = np.square(anchor - negative)

        if dist == 'euclidean':
            positive_distance = np.sqrt(np.sum(positive_distance, axis=1, keepdims=True))
            negative_distance = np.sqrt(np.sum(negative_distance, axis=1, keepdims=True))
        elif dist == 'sqeuclidean':
            positive_distance = np.sum(positive_distance, axis=1, keepdims=True)
            negative_distance = np.sum(negative_distance, axis=1, keepdims=True)
        loss = positive_distance - negative_distance

        if margin == 'maxplus':
            loss = np.maximum(0, 1 + loss)
        elif margin == 'softplus':
            loss = np.log(1 + np.exp(loss))

        return loss.reshape((128,))
    
    return curry

def triplet_loss(inputs, dist='sqeuclidean', margin='maxplus'):
    anchor, positive, negative = inputs
    positive_distance = K.square(anchor - positive)
    negative_distance = K.square(anchor - negative)
    if dist == 'euclidean':
        positive_distance = K.sqrt(K.sum(positive_distance, axis=1, keepdims=True))
        negative_distance = K.sqrt(K.sum(negative_distance, axis=1, keepdims=True))
    elif dist == 'sqeuclidean':
        positive_distance = K.sum(positive_distance, axis=1, keepdims=True)
        negative_distance = K.sum(negative_distance, axis=1, keepdims=True)
    loss = positive_distance - negative_distance
    if margin == 'maxplus':
        loss = K.maximum(0.0, 1 + loss)
    elif margin == 'softplus':
        loss = K.log(1 + K.exp(loss))
    return K.mean(loss)

In [193]:
# to choose between functions
from tensorflow.contrib.losses.metric_learning import triplet_semihard_loss

n = 128
shape = [n, 3, 16]
x = np.random.rand(*shape)
f = TripletLoss()
start = time()
print(f(x).shape)
np_speed = round(n/(time()-start),2)
print(f'NP speed {np_speed} it/s')

# now for verification
for dist in ['sqeuclidean', 'euclidean']:
    for margin in ['maxplus', 'softplus']:
        f = TripletLoss(dist,margin)
        lhs, rhs = f(x[:200]), f(x)[:200]
        print(np.array_equiv(lhs, rhs), end=" ")

UnboundLocalError: local variable 'negative_distance' referenced before assignment

In [196]:
model = keras.models.Sequential([
    #first convolution
    Conv2D(16, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPool2D(2,2),
    #second
    Conv2D(32, (3,3), activation='relu'),
    MaxPool2D(2,2),
    #third
    Conv2D(64, (3,3), activation='relu'),
    MaxPool2D(2,2),
    #fourth
    Conv2D(64, (3,3), activation='relu'),
    MaxPool2D(2,2),
    #fifth
    Conv2D(64, (3,3), activation='relu'),
    MaxPool2D(2,2),
    #hidden
    Flatten(),
    Dense(4096, activation='relu'), #activity_regularizer=regularizers.l2(0.01)),
    #Last layer
    Dense(16)
])
from tensorflow.keras.optimizers import RMSprop
model.compile(loss=triplet_loss, optimizer=RMSprop(lr=0.001), metrics=['acc'])
model.summary()

TypeError: Tensor objects are only iterable when eager execution is enabled. To iterate over this tensor use tf.map_fn.